In [1]:
import warnings
warnings.filterwarnings("ignore")

from prop_comparison_methods_beta import *

## Upload data and preprocess

First we specify some settings for this notebook

In [2]:
#first we select the genotype 
genotype = 'ecadGFPnbG4'
#genotype = 'ecadGFPnbG4myoVI'

#some lists and dicts that we refer to later

#update names of devstages
devstage_map = {
                "96hAEL":"96hAEL",
                "120hAEL":"120hAEL",
                "upcrawling":"wL3",
                "whitePupa":"0hAPF",
                "2hAPF":"2hAPF",
                "4hAPF":"4hAPF",
                "6hAPF":"6hAPF",
               }

#declare colors for each devstage and crosssection and region
color_dict = {
              '96hAEL':'#f1ef81',
              '120hAEL':'#efa636',
              'wL3':'#414243',
              '0hAPF':'#7d99cd', 
              '2hAPF':'#64a9dd', 
              '4hAPF':'#78cfdb',
              '6hAPF':'#71c382',
              'DV' : 'purple',
              'outDV' : 'green',
             }

columns = ['devstage', 'discName', 'region', 'k_dist', 'roi',
           'area','neighbour_number','elongation_tensor_norm_max', 
           'Qrr_geom_inPlane', 'Qphiphi', 'Qnn', 'Qrphi', 'Qrn', 'Qphin',
           'countInBin', 'cumcount',
           'k_dist_pathlength', #'k_dist_pathlength_poly' 
          ]

rois = ['outDV', 'DV']
devstages = [#"96hAEL",
             "wL3","0hAPF","2hAPF","4hAPF", "6hAPF",
]

############################################
# Dictionary of pairs of stages to compare #
############################################

#for the simulations, we use the cumulative version
devstage_combinations = pd.DataFrame({'devstage_init':[
                                                       #'wL3','0hAPF','2hAPF','4hAPF' #diff between consecutive stages
                                                       'wL3','wL3','wL3', #'wL3' #cumulative
    
                                                      ],
                                     'devstage_final':[
                                                       #'4hAPF',
                                                       '0hAPF','2hAPF','4hAPF',#'6hAPF' 
                                                      ],}
                                    )

Next we read the data

In [3]:
#upload data
df = pd.read_pickle('../data/DFallDiscsIncreaselimitcounts.pkl')
#df = pd.read_pickle('../data/DFallDiscslimitcounts.pkl')
df = df[df['genotype'] == genotype]
df["devstage"] = [devstage_map[x] for x in df["devstage"].values]
df = df[columns]

In [4]:
df.head()

,devstage,discName,region,k_dist,roi,area,neighbour_number,elongation_tensor_norm_max,Qrr_geom_inPlane,Qphiphi,Qnn,Qrphi,Qrn,Qphin,countInBin,cumcount,k_dist_pathlength
0,96hAEL,20220517_ecadGFPnbG4_96hAEL_disc8_outDV,outDV,13,dorsal,1.646873,4,0.079496,-0.065859,0.056459,0.009400,0.031090,0.003897,0.033033,42.0,288.0,28.560460
1,96hAEL,20220517_ecadGFPnbG4_96hAEL_disc8_outDV,outDV,13,dorsal,6.991516,7,0.297667,-0.280260,0.288938,-0.008678,-0.057726,-0.009983,0.029122,42.0,288.0,31.961953
2,96hAEL,20220517_ecadGFPnbG4_96hAEL_disc8_outDV,outDV,13,dorsal,9.529885,6,0.178425,-0.039228,0.051418,-0.012190,-0.165394,-0.004449,0.012512,42.0,288.0,32.296625
3,96hAEL,20220517_ecadGFPnbG4_96hAEL_disc8_outDV,outDV,13,dorsal,4.962586,5,0.329087,0.102858,-0.101544,-0.001314,0.311286,-0.010702,-0.012868,42.0,288.0,29.428472
4,96hAEL,20220517_ecadGFPnbG4_96hAEL_disc8_outDV,outDV,13,dorsal,5.680147,7,0.135623,0.052955,-0.049194,-0.003761,-0.122833,0.006176,-0.008721,42.0,288.0,30.195391


## Bin data within rings

In [5]:
#discName contains the name of the disc as well as region
groupby_cols = ['devstage', 'region', 'discName', 'k_dist']
#here we pool cells within a ring and calculate the mean
df_pool_k = df.groupby(groupby_cols).agg('mean').reset_index() 
#we offset k_dist values if the rings do not start with k_dits = 0
k_dist_offset = df.groupby(['discName']).k_dist.agg('min')#.reset_index().set_index('discName') #some discs can have k starting from non-zero value, so we offset them by the starting k
df_pool_k['k_dist'] = df_pool_k['k_dist'] - k_dist_offset[df_pool_k['discName'].values].values #offsetting k values


In [6]:
#compute Qnorm and exponential of Qnorm - to be used later

#function to compute area weighted average
wm = lambda x: np.average(x, weights=df.loc[x.index, "area"])

df_pool_areaWeighted_k = df.groupby(groupby_cols).agg(Qrr_geom_inPlane = pd.NamedAgg(column = 'Qrr_geom_inPlane', aggfunc = wm),
                                                      Qrphi = pd.NamedAgg(column = 'Qrphi', aggfunc = wm),
                                                      Qphiphi = pd.NamedAgg(column = 'Qphiphi', aggfunc = wm),
                                                     ).reset_index() 
df_pool_k[["Qrr_geom_inPlane", "Qrphi", "Qphiphi"]] = df_pool_areaWeighted_k[["Qrr_geom_inPlane", "Qrphi", "Qphiphi"]]

df_pool_k['Qnorm'] = np.sqrt(df_pool_k['Qrr_geom_inPlane']**2 + df_pool_k['Qrphi']**2)
df_pool_k['exp_signed_Qnorm'] = np.exp(np.sign(df_pool_k['Qrr_geom_inPlane'])*df_pool_k['Qnorm'])

## Average discs within devstage and region

In [7]:
groupby_cols = ['devstage', 'region', 'k_dist']
#here we pool discs within a devstage and calculate the mean and std
df_pool_devstage = df_pool_k.groupby(groupby_cols).agg(['mean', 'std']).reset_index()
colnames = [x[0]+'_'+x[1] if x[0] not in groupby_cols else x[0] for x in df_pool_devstage.columns]
df_pool_devstage.columns = colnames #removing multi-indexing

#compute Qnorm
df_pool_devstage['Qnorm_mean'] =  np.sqrt( df_pool_devstage['Qrr_geom_inPlane_mean']**2 + df_pool_devstage['Qrphi_mean']**2)#Norm of mean
df_pool_devstage['Qnorm_std'] = (df_pool_devstage['Qrr_geom_inPlane_mean']*df_pool_devstage['Qrr_geom_inPlane_std'] + df_pool_devstage['Qrphi_mean']*df_pool_devstage['Qrphi_std'])/df_pool_devstage['Qnorm_mean']

df_pool_devstage['exp_signed_Qnorm_mean'] = np.exp(np.sign(df_pool_devstage['Qrr_geom_inPlane_mean'])*df_pool_devstage['Qnorm_mean'])
df_pool_devstage['exp_signed_Qnorm_std'] = df_pool_devstage['exp_signed_Qnorm_mean']*df_pool_devstage['Qnorm_std']

df_pool_devstage.head()

,devstage,region,k_dist,area_mean,area_std,neighbour_number_mean,neighbour_number_std,elongation_tensor_norm_max_mean,elongation_tensor_norm_max_std,Qrr_geom_inPlane_mean,...,countInBin_mean,countInBin_std,cumcount_mean,cumcount_std,k_dist_pathlength_mean,k_dist_pathlength_std,Qnorm_mean,Qnorm_std,exp_signed_Qnorm_mean,exp_signed_Qnorm_std
0,0hAPF,DV,0,6.138776,1.899830,5.600000,0.565685,0.236052,0.074967,-0.057797,...,2.419048,0.626234,2.419048,0.626234,0.000000,0.000000,0.070636,-0.138054,0.931801,-0.128639
1,0hAPF,DV,1,7.049269,0.976519,5.859694,0.240154,0.214965,0.057095,-0.044956,...,5.163709,0.810137,7.296806,1.024164,2.740585,0.197002,0.049672,-0.009776,0.951542,-0.009302
2,0hAPF,DV,2,6.803410,0.972501,5.862647,0.250389,0.249500,0.057656,0.014085,...,6.821707,0.813107,13.939177,1.583098,5.952463,0.394122,0.017667,0.070687,1.017824,0.071946
3,0hAPF,DV,3,6.795584,1.264079,5.936568,0.161895,0.220404,0.025238,0.047206,...,6.948152,1.539218,20.804262,2.786416,9.463610,0.746017,0.047550,0.049418,1.048699,0.051824
4,0hAPF,DV,4,6.757545,1.057610,6.004347,0.113781,0.224231,0.026976,0.070369,...,7.368880,0.874314,27.901782,3.594859,12.758039,1.085376,0.070963,0.050287,1.073541,0.053985


---

#### Scale pathlengths

Not going to do this right now

---

## Compare discs between different stages to compute observed deformation

#### Area analysis

In [8]:
#function to compare between pairs of stages
prop = 'area'

[area_diff, area_diff_stat] = get_prop_differences(df_pool_k, prop = prop,operation = 'divide-sqrt',
                                                   devstage_combinations=devstage_combinations, 
                                                   fit_param = 'N_beta_mean'
                                                  )

In [11]:
area_diff_stat.head()

,k_beta,N_beta_mean,N_beta_std,area_diff_mean,area_diff_std,area_beta_mean,area_beta_std,roi,devstage_init,devstage_final,fit_area_diff,fit_area_coeffs
0,0,1.000000,0.000000,1.137680,0.245434,4.823720,1.970103,outDV,wL3,0hAPF,1.177328,"[-0.00013691937141762107, 1.1774649077498145]"
1,1,4.342857,0.284029,1.088623,0.144353,4.786623,0.504481,outDV,wL3,0hAPF,1.176870,"[-0.00013691937141762107, 1.1774649077498145]"
2,2,11.656190,0.808651,1.174478,0.195052,4.258522,0.469923,outDV,wL3,0hAPF,1.175869,"[-0.00013691937141762107, 1.1774649077498145]"
3,3,22.019617,1.290569,1.158371,0.155974,4.591164,0.460819,outDV,wL3,0hAPF,1.174450,"[-0.00013691937141762107, 1.1774649077498145]"
4,4,36.593492,0.637465,1.147090,0.143123,4.337342,0.493906,outDV,wL3,0hAPF,1.172455,"[-0.00013691937141762107, 1.1774649077498145]"


#### Elongationheadlysis

In [9]:
# use merge with previous dataframes
prop = 'exp_signed_Qnorm'

[el_diff, el_diff_stat] = get_prop_differences(df_pool_k, prop = prop,operation = 'divide',
                                                   devstage_combinations=devstage_combinations,
                                                   fit_param='N_beta_mean',
                                              )

In [12]:
el_diff_stat.head()

,k_beta,N_beta_mean,N_beta_std,exp_signed_Qnorm_diff_mean,exp_signed_Qnorm_diff_std,exp_signed_Qnorm_beta_mean,exp_signed_Qnorm_beta_std,roi,devstage_init,devstage_final,fit_exp_signed_Qnorm_diff,fit_exp_signed_Qnorm_coeffs
0,0,1.000000,0.000000,1.014674,0.130442,0.984330,0.093329,outDV,wL3,0hAPF,0.981648,"[-6.999623275268073e-05, 0.9817176262532283]"
1,1,4.342857,0.284029,1.061811,0.122598,0.959011,0.070707,outDV,wL3,0hAPF,0.981414,"[-6.999623275268073e-05, 0.9817176262532283]"
2,2,11.656190,0.808651,1.023568,0.105142,0.990760,0.070260,outDV,wL3,0hAPF,0.980902,"[-6.999623275268073e-05, 0.9817176262532283]"
3,3,22.019617,1.290569,1.082783,0.070269,0.970429,0.056948,outDV,wL3,0hAPF,0.980176,"[-6.999623275268073e-05, 0.9817176262532283]"
4,4,36.593492,0.637465,0.963976,0.056018,1.023199,0.044707,outDV,wL3,0hAPF,0.979156,"[-6.999623275268073e-05, 0.9817176262532283]"


#### Rearrangement analysis

In [10]:
# use merge with previous dataframes
[k_N_alldiscs, k_N_mean] = analyze_ring_cell_numbers(df, devstages = devstages)

[k_diff, k_diff_stat] = get_k_differences(k_N_alldiscs, devstage_combinations = devstage_combinations, rois = rois, fit_param = 'ref_pathlength_scaled_beta_mean', )
#N_ref_pathlength_dict = k_scaled_dist_dict


In [13]:
k_diff_stat.head()

,k_beta,N_beta_mean,N_beta_std,k_diff_mean,k_diff_std,DDk/DNDt_mean,DDk/DNDt_std,DDk/Dt_mean,DDk/Dt_std,lambda_rearrangement_mean,lambda_rearrangement_std,roi,devstage_init,devstage_final
0,1.0,4.3,0.248525,-0.165714,0.240309,0.007476,0.022920,0.060091,0.160579,1.060091,0.160579,outDV,wL3,0hAPF
1,2.0,11.4,0.497050,-0.105624,0.348514,0.001231,0.011577,0.014928,0.120291,1.014928,0.120291,outDV,wL3,0hAPF
2,3.0,21.8,1.044594,-0.090696,0.412386,0.004481,0.009001,0.069524,0.134795,1.069524,0.134795,outDV,wL3,0hAPF
3,4.0,36.4,0.591608,-0.021172,0.440483,0.001672,0.005010,0.035590,0.096006,1.035590,0.096006,outDV,wL3,0hAPF
4,5.0,55.1,1.617005,0.014417,0.448896,0.003378,0.004843,0.076149,0.108502,1.076149,0.108502,outDV,wL3,0hAPF


### Combine dataframes to get a single dataframe
A dataframe that has columns for 

#### Scale pathlength

In [ ]:
#normalize the pathlength by the pathlength for the k_max of the reference devstage
#this has been calculated earlier

# Visualization

#### pathlength analysis and N vs k

#### Area - absolute values

#### Area - lambda values

#### Elongation - absolute values

#### Elongation - lambda values

#### Rearrangement - lambda values